In [1]:
# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)
from urllib.parse import urlparse,parse_qs
# 生成實體物件
# line_bot_api = LineBotApi()
# handler = WebhookHandler()

In [2]:
!pip install redis

    100% |████████████████████████████████| 71kB 727kB/s ta 0:00:011
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
    SourceUser, SourceGroup, SourceRoom,
    TemplateSendMessage, ConfirmTemplate, MessageAction,
    ButtonsTemplate, ImageCarouselTemplate, ImageCarouselColumn, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    CarouselTemplate, CarouselColumn, PostbackEvent,
    StickerMessage, StickerSendMessage, LocationMessage, LocationSendMessage,
    ImageMessage, VideoMessage, AudioMessage, FileMessage,
    UnfollowEvent, FollowEvent, JoinEvent, LeaveEvent, BeaconEvent,
    FlexSendMessage, BubbleContainer, ImageComponent, BoxComponent,
    TextComponent, SpacerComponent, IconComponent, ButtonComponent,
    SeparatorComponent, QuickReply, QuickReplyButton
)
from linebot.models import (
    MessageEvent, PostbackEvent, 
    TextSendMessage, TemplateSendMessage,
    TextMessage, ButtonsTemplate,
    PostbackTemplateAction, MessageTemplateAction,
    URITemplateAction, 
)

In [4]:

import redis

#製作redis連線
redis = redis.Redis(
    #redis container的host name
    host='redis',
    port=6379,
    #預設沒密碼，成功之後設密碼
    password=None,
    #給格式
    charset="utf-8",
    #要解碼不然回傳的資料前面會多一個b
    decode_responses=True)


In [5]:
# 載入json處理套件
import json

# 載入基礎設定檔
secretFile=json.load(open("./secret_key",'r'))
server_url=secretFile.get("server_url")


# channel_access_token是用於傳送封包去給line的認證使用類似這個是私鑰，而公鑰已經丟到line那邊，拿著這個就可以有VIP特權
# secret_key是用於當line傳送封包過來時確定line是否為本尊，沒有被仿冒
line_bot_api = LineBotApi(secretFile.get("channel_access_token"))
handler = WebhookHandler(secretFile.get("secret_key"))
menu_id = secretFile.get("rich_menu_id")


In [6]:
'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
TextSendMessage(text="CC102-Line考古題機器人。")
]
    

In [7]:
"""

  啟用伺服器基本樣板啟用伺服器基本 

"""

# 引用Web Server套件
from flask import Flask, request, abort , jsonify



# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/")


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [8]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    ################### 須作例外處理
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
    # 將用戶資訊做成合適Json

    user_info = {  
        "user_open_id":user_profile.user_id,
        "user_nick_name":user_profile.display_name,
        "user_status" : "",
        "user_img" : user_profile.picture_url,
        "user_register_menu" : menu_id
    }
    #將json傳回API Server
    Endpoint='http://192.168.122.100:5001/users'   
    Header={'Content-Type':'application/json'}
    Response=requests.post(Endpoint,headers=Header,data=json.dumps(user_info))
    
    # 先暫時寫的，用來檢查錯誤情形，方便除錯
    print(Response)
    print(Response.text)
    
    # 將菜單綁定在用戶身上
    
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (user_profile.user_id, menu_id)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFile["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    print(lineLinkMenuResponse)
    print(lineLinkMenuResponse.text)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
         event.reply_token,
         reply_message_list
    )
    
    #再跟老師討論存在redis的值有沒有需要進去mysql
    redis.hmset(user_profile.user_id, {'result': 0,"total" : 0})
    
    
    

In [9]:
import random
#這個函式方便日後寫有關亂數的函式
def test_sys():
    #跑亂數
    a=random.sample(range(1,2), 1)
    #api server接口位置
    url =  'http://192.168.122.100:5001/question/sysops'
    #裝query string的部份
    payload = {'question_id' : a}
    #傳送封包
    a = requests.get(url,params=payload)
    #將回傳結果取出
    a=a.json()
    return a

In [17]:
#寫一個函式是看正解給result使用
def true_answer(a,answer):
    if a['true_answer']==answer:
        return 'True'
    else:
        return 'False'

def correct(a,answer):
    if a['true_answer']==answer:
        correct = 'Correct'+"\n"+a['true_answer_decribe_content']+"\n\n"+a['external_link']
        return correct
    else:
        wrong = "error!,Ans:"+a['true_answer']+"\n"+a['true_answer_decribe_content']+"\n\n"+a['external_link']
        return wrong

In [18]:
#做一個回傳dev考題的函式
def testsys():
    #只有sa的題目可用
    a = test_sys()
    #這邊使用quick reply的方式
    quickreply = TextSendMessage(
                text='Choose your answer',
                quick_reply=QuickReply(
                    items=[
                        QuickReplyButton(
                            #使用postback action類似按鈕的概念
                            action=PostbackAction(label="A",
                                                  #這邊使用true_answer()來幫助result
                                                  data="type=answer&result="+true_answer(a,'A'),
                                                  #使用correct()幫助回傳訊息
                                                  text=correct(a,'A')
                                                 )
                        ),
                        QuickReplyButton(
                            action=PostbackAction(label="B",
                                                  data="type=answer&result="+true_answer(a,'B'),
                                                  text=correct(a,'B')
                                                 )
                        ),
                        QuickReplyButton(
                            action=PostbackAction(label="C",
                                                  data="type=answer&result="+true_answer(a,'C'),
                                                  text=correct(a,'C')
                                                 )
                        ),
                        QuickReplyButton(
                            action=PostbackAction(label="D",
                                                  data="type=answer&result="+true_answer(a,'D'),
                                                  text=correct(a,'D')
                                                 )
                        )
                    ]))
    #全部回傳的list
    dev_reply_message_list = [
        TextSendMessage(text=a["question_content"]),
        TextSendMessage(text=a["answer1_content"]+"\n\n"+a["answer2_content"]+"\n\n"+a["answer3_content"]+"\n\n"+a["answer4_content"]),
        #回傳quick reply選單
        quickreply    
    ]
    return dev_reply_message_list

In [20]:


#用戶點擊button後，觸發postback event，對其回傳做相對應處理
@handler.add(PostbackEvent)
def handle_post_message(event):
    #抓取user資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
    #抓取postback action的data
    data = event.postback.data
    #用query string 解析data
    data=parse_qs(data)
    #出考題
    if (data['type']==['question']):
        #每要求出題後，redis 的total增加一
        redis.hincrby(user_profile.user_id,"total")
        if (data['question_type']==['sysops']):
            #回覆一組回覆串
            line_bot_api.reply_message(
            event.reply_token,
            testsys())
        elif (data['question_type']==['develop']):
            line_bot_api.reply_message(
            event.reply_token,
            testsys())
        elif (data['question_type']==['sa']):
            line_bot_api.reply_message(
            event.reply_token,
            testsys())
    #按回答後統計資料
    elif (data['type']==['answer']):
        if (data['result']==['True']):
            #每答對一題，redis的result增加一
            redis.hincrby(user_profile.user_id,"result")
            
    else:
        pass


In [19]:
'''

當用戶發出文字消息時，判斷文字內容是否包含[::text:]，
    若有，則從template_message_dict 內找出相關訊息
    若無，則回傳預設訊息。

'''

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
#將這次event的參數抓進來
def handle_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    #總答對題數
    correct = redis.hget(user_profile.user_id,"result")
    #總回答題數
    total = redis.hget(user_profile.user_id,"total")
    #回覆的訊息串
    line_bot_api.reply_message(
    event.reply_token,
    TextSendMessage(text="總共答對 (%s)題\n總共回答 (%s)題" % (correct,total)))

In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0',port=5000)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.19.0.4 - - [18/Sep/2018 10:22:36] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:22:47] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:22:51] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:23:19] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:23:29] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:23:37] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:24:08] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:24:29] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:24:37] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:24:46] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:25:38] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:35:24] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:37:37] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:37:44] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 10:37:52] "POST / HTTP/1.1" 200 -
172.19.0.4 -

<Response [200]>
{
  "status_describe": "success add user"
}

<Response [200]>
{}


172.19.0.4 - - [18/Sep/2018 11:07:44] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 11:08:02] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 11:08:40] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 11:09:00] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 11:09:45] "POST / HTTP/1.1" 200 -
172.19.0.4 - - [18/Sep/2018 11:13:17] "POST / HTTP/1.1" 200 -
